In [1]:
# jobkorea_passassay_by_schpart_auto.py
import asyncio, re, time
from typing import List, Dict, Any, Optional, Tuple
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import pandas as pd
from playwright.async_api import TimeoutError as PwTimeoutError

# ===== 설정 =====
LIST_URL_TPL = ("https://www.jobkorea.co.kr/starter/PassAssay"
                "?schPart=100{code}&schWork=&schEduLevel=&schCType=&schGroup="
                "&isSaved=1&isFilterChecked=1&OrderBy=0&schTxt=")
START_CODE = 26   # 10026부터
END_CODE   = 46   # 10046까지 (포함)
NAV_TIMEOUT = 45000
STEP_TIMEOUT = 20000
PAGES_PER_PART = 1  # 요청: 한 파트당 목록 1페이지만 처리
CONCURRENCY = 1     # 상세는 하나씩 순차 처리

# =============== 유틸 ===============
def clean(s: Optional[str]) -> str:
    return re.sub(r"\s+", " ", (s or "")).strip()

def slug_for_filename(s: str) -> str:
    s = re.sub(r"[ \t\-\_·∙•\.,/]+", "", s or "").strip()
    s = re.sub(r"[^0-9A-Za-z가-힣]+", "", s)
    return s or "all"

# ---------- 안전한 페이지 이동 ----------
async def safe_goto(page, url: str, css_to_wait: str = "body") -> bool:
    try:
        await page.goto(url, wait_until="domcontentloaded", timeout=NAV_TIMEOUT)
        if css_to_wait:
            await page.wait_for_selector(css_to_wait, timeout=STEP_TIMEOUT)
        return True
    except PwTimeoutError:
        pass
    try:
        await page.goto(url, wait_until="commit", timeout=20000)
        if css_to_wait:
            await page.wait_for_selector(css_to_wait, timeout=STEP_TIMEOUT)
        return True
    except PwTimeoutError:
        pass
    try:
        await page.reload(wait_until="domcontentloaded", timeout=NAV_TIMEOUT)
        if css_to_wait:
            await page.wait_for_selector(css_to_wait, timeout=STEP_TIMEOUT)
        return True
    except Exception:
        return False

# ---------- 무거운 리소스 차단(속도/안정성) ----------
async def block_heavy_resources(ctx):
    async def handler(route):
        rt = route.request.resource_type
        if rt in {"image", "media", "font", "stylesheet"}:
            await route.abort()
        else:
            await route.continue_()
    await ctx.route("**/*", handler)

# =============== QnA 추출 ===============
Q_PAT = re.compile(r"^(?:Q[\.\):\s]|질문\b|Question\b)", re.I)
A_PAT = re.compile(r"^(?:A[\.\):\s]|답변\b|Answer\b)", re.I)

def uniq_qna(pairs):
    seen, out = set(), []
    for q, a in pairs:
        qn, an = clean(q), clean(a)
        key = (qn.lower(), an.lower())
        if qn and an and key not in seen:
            seen.add(key); out.append((qn, an))
    return out

def extract_qna_all(soup: BeautifulSoup):
    pairs = []
    # 1) <dl> dt(Q)/dd(A)
    for dl in soup.find_all("dl"):
        for dt in dl.find_all("dt"):
            t = clean(dt.get_text(" ", strip=True))
            if not (Q_PAT.search(t) or "질문" in t or t.startswith("Q")):
                continue
            dd = dt.find_next_sibling("dd")
            if not dd:
                continue
            q = Q_PAT.sub("", t).strip() or t
            a = A_PAT.sub("", clean(dd.get_text(" ", strip=True))).strip()
            if q and a:
                pairs.append((q, a))
    # 2) 클래스 기반
    Q_SEL = [".q",".qbox",".qBox",".qTit",".qTitle",".q-title",".q_txt",".q-txt",
             ".question",".qCont",".qcontent",".qna-q",".qna_q"]
    A_SEL = [".a",".abox",".aBox",".aTit",".a_txt",".a-txt",
             ".answer",".aCont",".acontent",".qna-a",".qna_a"]
    q_nodes = []
    for sel in Q_SEL:
        q_nodes += soup.select(sel)
    for qn in q_nodes:
        q = Q_PAT.sub("", clean(qn.get_text(" ", strip=True))).strip()
        sib, a_txt = qn.find_next_sibling(), ""
        while sib and not a_txt:
            classes = " ".join(sib.get("class", [])).lower()
            if any(c.replace(".","") in classes for c in A_SEL):
                a_txt = clean(sib.get_text(" ", strip=True)); break
            sib = sib.find_next_sibling()
        if not a_txt:
            for sel in A_SEL:
                cand = qn.find_next(sel)
                if cand:
                    a_txt = clean(cand.get_text(" ", strip=True)); break
        if q and a_txt:
            a = A_PAT.sub("", a_txt).strip()
            pairs.append((q, a))
    # 3) 헤딩 휴리스틱
    if not pairs:
        for h in soup.find_all(["strong","b","h2","h3","h4","h5","p","span"]):
            t = clean(h.get_text(" ", strip=True))
            if not t or not (Q_PAT.search(t) or t.startswith("Q") or "질문" in t):
                continue
            q = Q_PAT.sub("", t).strip() or t
            sib, a, limit = h.find_next_sibling(), "", 6
            while sib and limit > 0 and not a:
                s = clean(sib.get_text(" ", strip=True))
                if s:
                    if A_PAT.search(s):
                        a = A_PAT.sub("", s).strip(); break
                    if len(s) > 10:
                        a = s; break
                sib = sib.find_next_sibling(); limit -= 1
            if q and a:
                pairs.append((q, a))
    return uniq_qna(pairs)

# =============== selfTopBx: 회사/포지션 ===============
PERIOD_POS_PAT = re.compile(r"^\s*(\d{4}년(?:\s*(?:상반기|하반기|정기|수시|동계|하계))?)\s+(.+)$")
def extract_company_position(soup: BeautifulSoup) -> Tuple[str, str]:
    # 회사
    company = ""
    for sel in [".selfTopBx h2.hd strong a",".selfTopBx h2.hd strong",
                ".selfTopBx .coName",".selfTopBx .assayCo",".selfTopBx .corpName"]:
        n = soup.select_one(sel)
        if n and n.get_text(strip=True):
            company = clean(n.get_text(" ", strip=True)); break
    # 포지션(em에서 Period/position 분리 → position만 사용)
    position = ""
    box = soup.select_one(".selfTopBx")
    em = box.select_one("em") if box else None
    txt = clean(em.get_text(" ", strip=True)) if em else ""
    if txt:
        m = PERIOD_POS_PAT.match(txt)
        if m:
            position = clean(m.group(2))
        else:
            position = txt
    return company, position

# =============== 목록 1페이지 상세 URL 수집 ===============
async def collect_urls_from_list(page) -> List[str]:
    urls = []
    try:
        await page.wait_for_selector(
            'a[href*="/starter/PassAssay/"], a[href*="/Starter/PassAssay/"]',
            timeout=STEP_TIMEOUT
        )
    except:
        pass
    hrefs = await page.eval_on_selector_all(
        'a[href*="/starter/PassAssay/"], a[href*="/Starter/PassAssay/"]',
        'els => els.map(e => e.getAttribute("href"))'
    )
    for h in hrefs or []:
        if not h: 
            continue
        full = urljoin(page.url, h)
        if any(k in full.lower() for k in ["/view","passassay/view","no="]):
            urls.append(full)
    # dedup
    seen, out = set(), []
    for u in urls:
        if u not in seen:
            seen.add(u); out.append(u)
    return out

# =============== 상세 파싱(순차) ===============
async def parse_detail(ctx, url: str) -> Dict[str, Any]:
    page = await ctx.new_page()
    page.set_default_navigation_timeout(NAV_TIMEOUT)
    page.set_default_timeout(STEP_TIMEOUT)
    try:
        ok = await safe_goto(page, url, css_to_wait="body")
        if not ok:
            return {"ok": False, "error": "goto-timeout"}
        html = await page.content()
        soup = BeautifulSoup(html, "lxml")
        company, position = extract_company_position(soup)
        qna_pairs = extract_qna_all(soup)
        qna = [{"q": q, "a": a} for q, a in qna_pairs]
        return {"ok": True, "company": company, "position": position, "qna": qna}
    except Exception as e:
        return {"ok": False, "error": f"{type(e).__name__}: {e}"}
    finally:
        await page.close()

# =============== 메인 파이프라인 ===============
async def run_by_schpart_codes():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        ctx = await browser.new_context(
            user_agent=("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                        "(KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"),
            locale="ko-KR",
            timezone_id="Asia/Seoul",
            viewport={"width": 1366, "height": 900}
        )
        await ctx.add_init_script(
            "Object.defineProperty(navigator, 'webdriver', {get: () => undefined});"
        )
        await block_heavy_resources(ctx)

        list_page = await ctx.new_page()
        list_page.set_default_navigation_timeout(NAV_TIMEOUT)
        list_page.set_default_timeout(STEP_TIMEOUT)

        # 10026 ~ 10046 순회
        for code in range(START_CODE, END_CODE + 1):
            list_url = LIST_URL_TPL.format(code=code)
            print(f"\n=== schPart=100{code} 목록 진입 ===")
            ok = await safe_goto(list_page, list_url, css_to_wait="body")
            if not ok:
                print(f"  ❌ 목록 진입 실패: {list_url}")
                # 실패해도 다음 코드로 진행
                continue

            # (요청) 각 파트는 목록 1페이지만
            detail_urls = await collect_urls_from_list(list_page)
            print(f"  - 상세 URL {len(detail_urls)}개 발견")

            rows = []
            for i, u in enumerate(detail_urls, 1):
                print(f"    • ({i}/{len(detail_urls)}) {u}")
                r = await parse_detail(ctx, u)
                if not r.get("ok"):
                    print(f"      -> FAIL: {r.get('error')}")
                    continue
                company, position = r.get("company", ""), r.get("position", "")
                for qa in r.get("qna", []):
                    rows.append({
                        "company": company,
                        "position": position,
                        "q": qa.get("q", ""),
                        "a": qa.get("a", ""),
                    })

            # 파트별 CSV 저장
            df = pd.DataFrame(rows, columns=["company","position","q","a"])
            out = f"passassay_schPart_100{code}.csv"
            df.to_csv(out, index=False, encoding="utf-8-sig")
            print(f"  -> 저장 완료: {out} (행 {len(df)})")

        await browser.close()
        print("\n✅ 모든 schPart 처리 완료")

# =============== 엔트리포인트 ===============
if __name__ == "__main__":
    # Jupyter/IDE 호환
    try:
        loop = asyncio.get_running_loop()
        import nest_asyncio; nest_asyncio.apply()
        loop.run_until_complete(run_by_schpart_codes())
    except RuntimeError:
        asyncio.run(run_by_schpart_codes())



=== schPart=10026 목록 진입 ===
  - 상세 URL 20개 발견
    • (1/20) https://www.jobkorea.co.kr/starter/PassAssay/View/242004?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10026&Pass_An_Stat=0
    • (2/20) https://www.jobkorea.co.kr/starter/PassAssay/View/241949?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10026&Pass_An_Stat=0
    • (3/20) https://www.jobkorea.co.kr/starter/PassAssay/View/241903?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10026&Pass_An_Stat=0
    • (4/20) https://www.jobkorea.co.kr/starter/PassAssay/View/241724?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10026&Pass_An_Stat=0
    • (5/20) https://www.jobkorea.co.kr/starter/PassAssay/View/241750?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10026&Pass_An_Stat=0
    • (6/20) https://www.jobkorea.co.kr/starter/PassAssay/View/241751?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10026&Pass_An_Stat=0
    • (7/20) https://www.jobkorea.co.kr/starter/PassAssay/View/241844?Page=1&OrderBy=0&FavorCo_Stat=0&schPart=10026&Pass_An_Stat=0
    • (8/20) https://www.jobkorea.co